In [1]:
!pip install redis requests beautifulsoup4 sentence-transformers langchain


  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
INFO: pip is looking at multiple versions of sentence-transformers to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of sentence-transformers to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... error
  error: subprocess-exited-with-error
  
  × Preparing metadata (pypro

In [2]:
pip install -U langchain-community redis


  Using cached langchain-0.3.3-py3-none-any.whl.metadata (7.1 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.8/15.8 MB 16.4 MB/s eta 0:00:00a 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached httpx-0.27.2-py3-none-any.whl.metadata (7.1 kB)
  Using cached charset_normalizer-3.4.0-cp313-cp313-macosx_11_0_arm64.whl.metadata (34 kB)
  Using cached idna-3.10-py3-none-any.whl.metadata (10 kB)
  Using cached urllib3-2.2.3-py3-none-any.whl.metadata (6.5 kB)
  Using cached certifi-2024.8.30-py3-none-any.whl.metadata (2.2 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 15.7 MB/s eta 0:00:00
Using cached requests-2.32.3-py3-none-any.whl (64 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1

In [ ]:
import redis
import requests
from bs4 import BeautifulSoup
from sentence_transformers import SentenceTransformer
from langchain_community.vectorstores.redis import Redis
from langchain.embeddings import HuggingFaceEmbeddings

# Step 1: Connect to Redis Cloud (Replace with your Redis Cloud credentials)
redis_host = "redis-14406.c259.us-central1-2.gce.redns.redis-cloud.com"
redis_port = 14406  # Replace with your port
redis_password = "daZbJ7qv45e3WBwo8yzrSI8rXuN4foHX"  # Replace with your password

redis_conn = redis.StrictRedis(
    host=redis_host,
    port=redis_port,
    password=redis_password,
    decode_responses=True
)
redis_url = f"redis://:{redis_password}@{redis_host}:{redis_port}"

# Step 2: Initialize the embedding model and vector store
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vector_store = Redis(
    redis_url=redis_url,
    embedding=embedding_model,
    index_name="umiacs"  # Optional: Specify an index name
)


# Scraping all the links through one page: URL of the Special:AllPages page on the wiki

all_pages_url = 'https://wiki.umiacs.umd.edu/umiacs/index.php/Special:AllPages'
response = requests.get(all_pages_url)
soup = BeautifulSoup(response.content, 'html.parser')

# Create a set of unique pages on the wiki
pageset = set()
for link in soup.find_all('a'):
    href = link.get('href')
    # Ensure the link is valid and points to an internal page
    if href and href.startswith('/umiacs/index.php'):
        # Build the full URL for the wiki page
        full_url = 'https://wiki.umiacs.umd.edu' + href
        # Add the unique URL to the set, following redirects
        response = requests.get(full_url, allow_redirects=True)
        response_url = response.url
        pageset.add(response_url)

# List of URLs for HTML pages
html_links = list(pageset)


In [27]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")

MAX_TOKENS = 2048  # Max token limit for the model
OVERLAP = 100  # Token overlap for better context

# Step 2: Token-based chunking with overlap (ensures token limits are respected)
def chunk_text_with_overlap(text, max_tokens=MAX_TOKENS, overlap=OVERLAP):
    """Chunks text within token limits with overlap for context retention."""
    tokens = tokenizer.tokenize(text)  # Tokenize the input text
    chunks = []

    # Create sliding window chunks with overlap
    for i in range(0, len(tokens), max_tokens - overlap):
        chunk_tokens = tokens[i:i + max_tokens]
        chunk = tokenizer.convert_tokens_to_string(chunk_tokens)
        chunks.append(chunk)

    return chunks

# Step 3: Function to fetch HTML content
def fetch_html_from_url(url):
    try:
        response = requests.get(url)
        response.raise_for_status()
        return response.text
    except requests.exceptions.RequestException as e:
        print(f"Failed to fetch {url}: {e}")
        return None

# Step 4: Chunk HTML content into smaller pieces
def chunk_text(text, chunk_size=500):
    words = text.split()
    for i in range(0, len(words), chunk_size):
        yield " ".join(words[i:i + chunk_size])

# Step 5: Process each page, generate embeddings, and store them in Redis
def process_and_store_page(url):
    html_content = fetch_html_from_url(url)
    if html_content:
        soup = BeautifulSoup(html_content, "html.parser")
        text_content = soup.get_text(separator=" ", strip=True)
        chunks = chunk_text_with_overlap(text_content)
        for chunk in chunks:
            embedding = embedding_model.embed_query(chunk)
            vector_store.add_texts([chunk], embeddings=[embedding])

# Step 6: Process all pages and store embeddings in Redis
for url in html_links:
    process_and_store_page(url)

print("All pages processed and embeddings stored in Redis!")



Token indices sequence length is longer than the specified maximum sequence length for this model (2931 > 512). Running this sequence through the model will result in indexing errors


Failed to fetch https://wiki.umiacs.umd.edu/umiacs/index.php/UMIACS:General_disclaimer: 404 Client Error: Not Found for url: https://wiki.umiacs.umd.edu/umiacs/index.php/UMIACS:General_disclaimer
Failed to fetch https://wiki.umiacs.umd.edu/umiacs/index.php/UMIACS:Privacy_policy: 404 Client Error: Not Found for url: https://wiki.umiacs.umd.edu/umiacs/index.php/UMIACS:Privacy_policy
All pages processed and embeddings stored in Redis!


In [29]:
# Generate query embedding
query = "sbatch jobs"
query_embedding = embedding_model.embed_query(query)  # Ensure this is a valid vector

# Step: Perform similarity search by vector
try:
    # Use the correct syntax for similarity search by vector
    results = vector_store.similarity_search_by_vector(embedding=query_embedding, k=5)

    # Correctly extract text from Document objects
    for doc in results:
        print(doc.page_content)  # Access the content of the Document object
except Exception as e:
    print(f"Error during similarity search: {e}")

: ss # sbatch - - qos = default # set qos, this will determine what resources can be requested # sbatch - - nodes = 2 # number of nodes to allocate for your job # sbatch - - ntasks = 4 # request 4 cpu cores be reserved for your node total # sbatch - - ntasks - per - node = 2 # request 2 cpu cores be reserved per node # sbatch - - mem = 1g # memory required by job ; if unit is not specified mb will be assumed. for multi - node jobs, this argument allocates this much memory * per node * srun - - nodes = 1 - - mem = 512m bash - c " hostname ; python3 - - version " & # use srun to invoke commands within your job ; using an ' & ' srun - - nodes = 1 - - mem = 512m bash - c " hostname ; python3 - - version " & # will background the process allowing them to run concurrently wait # wait for any background processes to complete # once the end of the batch script is reached your job allocation will be revoked another useful thing to know is that you can pass additional arguments into your sbatch 

### In case we want to delete the vectors

In [26]:
def delete_all_vectors(redis_conn, index_name):
    """Deletes all keys in the given Redis namespace."""
    try:
        # Use SCAN to retrieve all keys that belong to the index namespace
        keys = redis_conn.scan_iter(f"{index_name}:*")
        for key in keys:
            redis_conn.delete(key)  # Delete each key
        print(f"All vectors from index '{index_name}' have been deleted.")
    except Exception as e:
        print(f"Error deleting vectors: {e}")

# Example usage
delete_all_vectors(redis_conn, "umiacs")

All vectors from index 'umiacs' have been deleted.
